In [12]:
import pandas as pan;
import numpy as np;
import matplotlib.pyplot as plotter;
import seaborn as sns;
from sklearn.preprocessing import StandardScaler;
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV;
from sklearn.linear_model import Ridge, ElasticNet;
from sklearn.metrics import mean_absolute_error, mean_squared_error;

In [2]:
matrix_data = pan.read_csv("Advertising.csv")
features = matrix_data.drop("sales", axis = 1)
label = matrix_data["sales"]

In [3]:
#train-test_split and scaling
feature_train, feature_test, label_train, label_test = train_test_split(features, label, test_size = 0.3, random_state = 42)
scaler = StandardScaler()
scaler.fit(feature_train)
feature_train = scaler.transform(feature_train)
feature_test = scaler.transform(feature_test)
learner = Ridge(alpha = 100)

In [7]:
scores = cross_val_score(learner, feature_train, label_train, scoring = "neg_mean_squared_error", n_jobs = 4, cv = 10)
scores
#then we can tune the alpha hyperparameter from here. Lets try cross-validate

array([ -4.72138179,  -8.13155956,  -5.43081344,  -6.61295023,
        -9.12876411,  -6.55443421,  -2.89336455, -12.05836429,
       -12.81830605,  -2.58786727])

In [9]:
extra_scores = cross_validate(learner, feature_train, label_train, scoring = ["neg_mean_squared_error", "neg_mean_absolute_error"], n_jobs = 4, cv = 15)
extra_scores = pan.DataFrame(extra_scores)
extra_scores["test_neg_mean_squared_error"].mean()

-6.9012042670944425

In [17]:
#initialize an elastic net model, without hyperparameters
elastic_learner = ElasticNet()
#put the adjustible hyperparameters in a dictionary
params = {"alpha" : [0.01, 0.1, 1, 2, 5, 10, 100], "l1_ratio": [0, 0.1, 0.5, 0.7, 0.9, 0.95, 1]}
gridder = GridSearchCV(estimator = elastic_learner, param_grid = params, scoring = ["neg_mean_squared_error", "neg_mean_absolute_error"], cv = 5, n_jobs = -1, verbose = 3, refit = "neg_mean_squared_error")

In [18]:
gridder.fit(feature_train, label_train)

Fitting 5 folds for each of 49 candidates, totalling 245 fits


GridSearchCV(cv=5, estimator=ElasticNet(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.1, 1, 2, 5, 10, 100],
                         'l1_ratio': [0, 0.1, 0.5, 0.7, 0.9, 0.95, 1]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'],
             verbose=3)

In [19]:
gridder.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=1)

In [21]:
print(gridder.best_params_)
print(gridder.best_estimator_)
print(gridder.best_score_)
gridder.best_index_

{'alpha': 0.1, 'l1_ratio': 1}
ElasticNet(alpha=0.1, l1_ratio=1)
-2.76568723225508


13

In [22]:
pan.DataFrame(gridder.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_neg_mean_squared_error,split1_test_neg_mean_squared_error,split2_test_neg_mean_squared_error,...,std_test_neg_mean_squared_error,rank_test_neg_mean_squared_error,split0_test_neg_mean_absolute_error,split1_test_neg_mean_absolute_error,split2_test_neg_mean_absolute_error,split3_test_neg_mean_absolute_error,split4_test_neg_mean_absolute_error,mean_test_neg_mean_absolute_error,std_test_neg_mean_absolute_error,rank_test_neg_mean_absolute_error
0,0.082599,4.883440e-04,0.012733,0.000000e+00,0.01,0,"{'alpha': 0.01, 'l1_ratio': 0}",-2.777719,-1.591078,-2.445888,...,1.110155,8,-1.280944,-1.038791,-1.215965,-1.310105,-1.286957,-1.226552,0.098949,10
1,0.049798,3.984832e-02,0.008238,5.513726e-03,0.01,0.1,"{'alpha': 0.01, 'l1_ratio': 0.1}",-2.775631,-1.591434,-2.443556,...,1.111097,7,-1.280474,-1.038503,-1.215051,-1.309766,-1.287773,-1.226313,0.099070,9
2,0.001596,4.887921e-04,0.000798,3.989713e-04,0.01,0.5,"{'alpha': 0.01, 'l1_ratio': 0.5}",-2.767528,-1.593222,-2.434443,...,1.112027,6,-1.278578,-1.037371,-1.211677,-1.308403,-1.290437,-1.225293,0.099489,8
3,0.001397,4.897266e-04,0.001197,3.985446e-04,0.01,0.7,"{'alpha': 0.01, 'l1_ratio': 0.7}",-2.763633,-1.594275,-2.429979,...,1.112047,5,-1.277616,-1.036807,-1.210009,-1.307715,-1.291683,-1.224766,0.099697,7
4,0.001595,4.862652e-04,0.000998,2.773865e-06,0.01,0.9,"{'alpha': 0.01, 'l1_ratio': 0.9}",-2.759844,-1.593398,-2.425577,...,1.112486,4,-1.276645,-1.035700,-1.208332,-1.307021,-1.292933,-1.224126,0.100118,5
5,0.001197,3.990426e-04,0.001397,4.887364e-04,0.01,0.95,"{'alpha': 0.01, 'l1_ratio': 0.95}",-2.758914,-1.592450,-2.424486,...,1.112751,3,-1.276400,-1.035226,-1.207911,-1.306847,-1.293247,-1.223926,0.100298,4
6,0.001400,4.868380e-04,0.000798,3.988801e-04,0.01,1,"{'alpha': 0.01, 'l1_ratio': 1}",-2.757991,-1.591506,-2.423400,...,1.113015,2,-1.276155,-1.034752,-1.207490,-1.306672,-1.293560,-1.223726,0.100479,3
7,0.002390,4.899551e-04,0.000998,1.181556e-06,0.1,0,"{'alpha': 0.1, 'l1_ratio': 0}",-2.959467,-1.856438,-2.892866,...,1.018984,14,-1.324112,-1.131610,-1.335573,-1.320856,-1.206461,-1.263722,0.081002,14
8,0.000998,6.319951e-04,0.000795,3.975313e-04,0.1,0.1,"{'alpha': 0.1, 'l1_ratio': 0.1}",-2.920581,-1.842830,-2.856136,...,1.026090,13,-1.311410,-1.127666,-1.326127,-1.317382,-1.210928,-1.258703,0.077744,13
9,0.000797,3.985182e-04,0.000399,4.890457e-04,0.1,0.5,"{'alpha': 0.1, 'l1_ratio': 0.5}",-2.777186,-1.804157,-2.716130,...,1.057770,12,-1.267108,-1.111959,-1.289823,-1.303973,-1.237955,-1.242164,0.068826,12


In [ ]:
#unlike the cross-val brothers, the gridders are the best fit model, thus they can predict like one
